<a href="https://colab.research.google.com/github/Ronak1998/Raspberrypi/blob/master/House_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# univariate cnn lstm example
import tensorflow.compat.v1 as tf
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import keras.utils

In [0]:
import pandas as pd
import time
from datetime import date, time, timedelta
import numpy as np
from numpy import asarray
from numpy import save
from sklearn import preprocessing
from google.colab import drive

In [45]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [46]:
file = r'/gdrive/My Drive/Colab Notebooks/NILM/REDDhouse1_lowf_VA.csv'
df = pd.read_csv(file)
print(df)

        TimeStamp    MAIN   OVEN   REFG  ...   BATH   HEAT   STOV   DIFF
0       1303132933    340      0      6  ...      1      0      0     79
1       1303132936    342      0      6  ...      1      0      0     81
2       1303132940    341      0      6  ...      1      0      0     79
3       1303132943    341      0      6  ...      1      0      0     79
4       1303132946    340      0      6  ...      1      0      0     79
...            ...    ...    ...    ...  ...    ...    ...    ...    ...
406743  1306266980    273      0    186  ...      1      0      0     31
406744  1306266983    273      0    187  ...      1      0      0     33
406745  1306266987    273      0    190  ...      1      0      0     30
406746  1306266990    273      0    189  ...      1      0      0     31
406747  1306266994    273      0    186  ...      1      0      0     32

[406748 rows x 13 columns]


In [47]:
array = df.to_numpy()
dataset = pd.DataFrame({'timestamp': array[:,0], 'Main': array[:,1], 'Oven': array[:,2], 'Refg': array[:,3], 'Dish': array[:,4], 'Kitch': array[:,5], 'Lite': array[:,6], 'Dryr': array[:,7], 'Micr': array[:,8], 'Bath': array[:,9], 'Heat': array[:,10], 'Stov': array[:,11],'Diff': array[:,12]})
dataset = dataset.set_index(['timestamp'])
dataset.index = pd.to_datetime(dataset.index, unit='s')
dataset = dataset.resample('1min').mean()
dataset = dataset.dropna()
dataset.describe()

,Main,Oven,Refg,Dish,Kitch,Lite,Dryr,Micr,Bath,Heat,Stov,Diff
count,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000,26297.000000
mean,382.908284,13.731572,54.990180,25.396969,57.172662,74.117537,35.850292,22.183838,7.003494,0.127257,0.114912,92.219572
std,747.425265,211.077353,85.166772,149.214637,87.244082,64.251739,288.437464,147.068870,91.989994,0.946020,0.799245,334.331542
min,90.666667,0.000000,0.500000,0.000000,40.266667,0.333333,0.000000,1.687500,0.000000,0.000000,0.000000,0.800000
25%,136.875000,0.000000,6.312500,0.000000,43.125000,3.200000,0.000000,4.000000,1.000000,0.000000,0.000000,32.687500
50%,175.812500,0.000000,6.750000,0.000000,44.000000,81.500000,0.000000,4.000000,1.000000,0.000000,0.000000,37.375000
75%,349.312500,0.000000,101.750000,0.000000,46.000000,84.400000,0.000000,4.000000,1.000000,0.000000,0.000000,54.000000
max,11556.250000,4156.625000,437.800000,1153.066667,2725.600000,418.666667,3251.812500,1771.250000,1664.187500,19.600000,16.333333,3378.937500


In [0]:
def LSTM_pre_processing(dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0


  flag = 0
  for i in range(starting+1,ending):
    if dataset["Lite"][i] == 1:
      if dataset["Lite"][i-1] == 1:
        flag+=1
      else:
        flag = 1
    else:
      flag = 0
    dataset["Kitch"][i] = flag

  continues_active_sequence = []
  temp = []
  for i in range(starting, ending):
    if dataset["Kitch"][i] > 0:
      temp1 = []
      temp1.append(dataset["Kitch"][i])
      temp1.append(dataset[appliance][i])
      temp.append(temp1)
    else:
      if len(temp) > 0:
        continues_active_sequence.append(temp)
        temp = []


  train_x = []
  train_y = []
  for i in range(1,len(continues_active_sequence)):
    temp1 = [0,0,0,0,0]
    train_x.append(temp1)
    norm = (continues_active_sequence[i][0][1])
    train_y.append(norm)
    temp1 = [0,0,0,0]
    for j in range(0,len(continues_active_sequence[i])-1):
      norm1 = (continues_active_sequence[i][j][0])
      temp1.append(norm1)
      train_x.append(temp1)
      norm2 = (continues_active_sequence[i][j+1][1])
      train_y.append(norm2)
      temp3 = []
      for k in range(1,len(temp1)):
        temp3.append(temp1[k])
      temp1 = temp3

  LSTM_X = np.asarray(train_x)
  LSTM_y = np.asarray(train_y)

  return LSTM_X, LSTM_y;

In [0]:
def CNN_pre_processing (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  x = dataset.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  dataset1 = pd.DataFrame(x_scaled)


  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset1[0][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)

  test_array = []
  for i in range(0,len(final_array)):
      temp = []
      for j in range(0,10):
          temp.append([0,0])
      for j in range(0,len(final_array[i])):
          temp.append(final_array[i][j])
      for j in range(0,10):
          temp.append([0,0])
      test_array.append(temp)



  training_set = []
  for i in range(0,len(test_array)):
      for j in range(0,len(test_array[i])-20):
          x_y = []
          temp = []
          for k in range(j,j+20):
              temp.append(test_array[i][k][0])
          x_y.append(temp)
          x_y.append(test_array[i][j+10][1])
          training_set.append(x_y)


  X = []
  y = []
  for i in range(0,len(training_set)):
    lets = training_set[i][0]
    lets = np.asarray(lets)
    X.append(lets)
    y.append(training_set[i][1])


  X = np.asarray(X)
  y = np.asarray(y)
  from keras.utils import to_categorical
  y = to_categorical(y)



  return X,y;

In [0]:
LSTM_X_refg,LSTM_y_refg = LSTM_pre_processing(dataset, "Refg", 0,  (len(dataset.index)-8000), 0, 100)

LSTM_X_micr,LSTM_y_micr = LSTM_pre_processing(dataset, "Micr", 0,  (len(dataset.index)-8000), 0, 600)

LSTM_X_dish1,LSTM_y_dish1 = LSTM_pre_processing(dataset, "Dish", 0,  (len(dataset.index)-8000), 100, 500)

LSTM_X_dish2,LSTM_y_dish2 = LSTM_pre_processing(dataset, "Dish", 0,  (len(dataset.index)-8000), 0, 850)

In [0]:
X_refg, y_refg = CNN_pre_processing(dataset, "Refg", 0, (len(dataset.index)-8000), 0, 100)

X_micr, y_micr = CNN_pre_processing(dataset, "Micr", 0, (len(dataset.index)-8000), 0, 600)

X_dish1, y_dish1 = CNN_pre_processing(dataset, "Dish", 0, (len(dataset.index)-8000), 100, 600)

X_dish2, y_dish2 = CNN_pre_processing(dataset, "Dish", 0, (len(dataset.index)-8000), 0, 850)

In [52]:
print(len(X_dish1), len(LSTM_X_dish1))

17740 357


In [0]:
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional

In [0]:
n_features = 1
#n_seq = 2
n_steps = 20
X_refg = X_refg.reshape((X_refg.shape[0], n_steps, n_features))

X_micr = X_micr.reshape((X_micr.shape[0], n_steps, n_features))

X_dish1 = X_dish1.reshape((X_dish1.shape[0], n_steps, n_features))

X_dish2 = X_dish2.reshape((X_dish2.shape[0], n_steps, n_features))


In [55]:
model_dish1 = Sequential()
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_dish1.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_dish1.add(MaxPooling1D(pool_size=2))
model_dish1.add(Flatten())
model_dish1.add(Dense(100, activation='relu'))
model_dish1.add(Dense(2, activation='softmax'))
model_dish1.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_dish1.fit(X_dish1, y_dish1, epochs=20, validation_split=0.1, verbose=1)

Train on 15966 samples, validate on 1774 samples
Epoch 1/20
15966/15966 [==============================] - 3s 211us/step - loss: 0.1065 - accuracy: 0.9771 - val_loss: 0.0225 - val_accuracy: 1.0000
Epoch 2/20
15966/15966 [==============================] - 3s 193us/step - loss: 0.0742 - accuracy: 0.9770 - val_loss: 0.0253 - val_accuracy: 1.0000
Epoch 3/20
15966/15966 [==============================] - 3s 196us/step - loss: 0.0684 - accuracy: 0.9773 - val_loss: 0.0108 - val_accuracy: 1.0000
Epoch 4/20
15966/15966 [==============================] - 3s 199us/step - loss: 0.0640 - accuracy: 0.9791 - val_loss: 0.0067 - val_accuracy: 0.9983
Epoch 5/20
15966/15966 [==============================] - 3s 196us/step - loss: 0.0607 - accuracy: 0.9798 - val_loss: 0.0123 - val_accuracy: 0.9972
Epoch 6/20
15966/15966 [==============================] - 3s 197us/step - loss: 0.0591 - accuracy: 0.9806 - val_loss: 0.0074 - val_accuracy: 1.0000
Epoch 7/20
15966/15966 [==============================] - 3s 19

In [56]:
model_dish2 = Sequential()
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_dish2.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_dish2.add(MaxPooling1D(pool_size=2))
model_dish2.add(Flatten())
model_dish2.add(Dense(100, activation='relu'))
model_dish2.add(Dense(2, activation='softmax'))
model_dish2.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_dish2.fit(X_dish2, y_dish2, epochs=20, validation_split=0.1, verbose=1)

Train on 15966 samples, validate on 1774 samples
Epoch 1/20
15966/15966 [==============================] - 4s 226us/step - loss: 0.0438 - accuracy: 0.9903 - val_loss: 3.5557e-04 - val_accuracy: 1.0000
Epoch 2/20
15966/15966 [==============================] - 3s 207us/step - loss: 0.0093 - accuracy: 0.9966 - val_loss: 1.5191e-04 - val_accuracy: 1.0000
Epoch 3/20
15966/15966 [==============================] - 3s 194us/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 3.3694e-04 - val_accuracy: 1.0000
Epoch 4/20
15966/15966 [==============================] - 3s 195us/step - loss: 0.0080 - accuracy: 0.9972 - val_loss: 2.4289e-05 - val_accuracy: 1.0000
Epoch 5/20
15966/15966 [==============================] - 3s 198us/step - loss: 0.0068 - accuracy: 0.9977 - val_loss: 8.5822e-04 - val_accuracy: 1.0000
Epoch 6/20
15966/15966 [==============================] - 3s 196us/step - loss: 0.0058 - accuracy: 0.9979 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 7/20
15966/15966 [===================

In [57]:
model_micr = Sequential()
model_micr.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_micr.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_micr.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_micr.add(MaxPooling1D(pool_size=2))
model_micr.add(Flatten())
model_micr.add(Dense(100, activation='relu'))
model_micr.add(Dense(2, activation='softmax'))
model_micr.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

model_micr.fit(X_micr, y_micr, epochs=20, validation_split=0.1, verbose=1)

Train on 15966 samples, validate on 1774 samples
Epoch 1/20
15966/15966 [==============================] - 3s 215us/step - loss: 0.0748 - accuracy: 0.9820 - val_loss: 0.0301 - val_accuracy: 0.9927
Epoch 2/20
15966/15966 [==============================] - 3s 199us/step - loss: 0.0404 - accuracy: 0.9844 - val_loss: 0.0197 - val_accuracy: 0.9961
Epoch 3/20
15966/15966 [==============================] - 3s 208us/step - loss: 0.0334 - accuracy: 0.9855 - val_loss: 0.0224 - val_accuracy: 0.9932
Epoch 4/20
15966/15966 [==============================] - 3s 208us/step - loss: 0.0315 - accuracy: 0.9862 - val_loss: 0.0222 - val_accuracy: 0.9938
Epoch 5/20
15966/15966 [==============================] - 3s 202us/step - loss: 0.0281 - accuracy: 0.9871 - val_loss: 0.0191 - val_accuracy: 0.9955
Epoch 6/20
15966/15966 [==============================] - 3s 208us/step - loss: 0.0268 - accuracy: 0.9882 - val_loss: 0.0179 - val_accuracy: 0.9961
Epoch 7/20
15966/15966 [==============================] - 3s 20

In [58]:
model_refg = Sequential()
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_steps,n_features)))
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_refg.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model.add(Dropout(0.5))
model_refg.add(MaxPooling1D(pool_size=2))
model_refg.add(Flatten())
model_refg.add(Dense(100, activation='relu'))
model_refg.add(Dense(2, activation='softmax'))
model_refg.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])


model_refg.fit(X_refg, y_refg, epochs=20, validation_split=0.1, verbose=1)



Train on 15966 samples, validate on 1774 samples
Epoch 1/20
15966/15966 [==============================] - 3s 213us/step - loss: 0.3634 - accuracy: 0.8146 - val_loss: 0.1984 - val_accuracy: 0.9092
Epoch 2/20
15966/15966 [==============================] - 3s 204us/step - loss: 0.2507 - accuracy: 0.8796 - val_loss: 0.2238 - val_accuracy: 0.9014
Epoch 3/20
15966/15966 [==============================] - 3s 202us/step - loss: 0.2304 - accuracy: 0.8938 - val_loss: 0.1654 - val_accuracy: 0.9177
Epoch 4/20
15966/15966 [==============================] - 3s 197us/step - loss: 0.2207 - accuracy: 0.8962 - val_loss: 0.1617 - val_accuracy: 0.9273
Epoch 5/20
15966/15966 [==============================] - 3s 196us/step - loss: 0.2113 - accuracy: 0.9010 - val_loss: 0.1415 - val_accuracy: 0.9363
Epoch 6/20
15966/15966 [==============================] - 3s 195us/step - loss: 0.2028 - accuracy: 0.9022 - val_loss: 0.1513 - val_accuracy: 0.9391
Epoch 7/20
15966/15966 [==============================] - 3s 19

In [0]:
n_steps_lstm = 5
n_features_lstm = 1
LSTM_X_refg = LSTM_X_refg.reshape((LSTM_X_refg.shape[0], LSTM_X_refg.shape[1], n_features_lstm))

LSTM_X_micr = LSTM_X_micr.reshape((LSTM_X_micr.shape[0], LSTM_X_micr.shape[1], n_features_lstm))

LSTM_X_dish1 = LSTM_X_dish1.reshape((LSTM_X_dish1.shape[0], LSTM_X_dish1.shape[1], n_features_lstm))

LSTM_X_dish2 = LSTM_X_dish2.reshape((LSTM_X_dish2.shape[0], LSTM_X_dish2.shape[1], n_features_lstm))

In [60]:
model_lstm_dish1 = Sequential()
model_lstm_dish1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_dish1.add(LSTM(50, activation='relu'))
model_lstm_dish1.add(Dense(1))
model_lstm_dish1.compile(optimizer='adam', loss='mse')

model_lstm_dish1.fit(LSTM_X_dish1, LSTM_y_dish1, epochs=30, verbose=1)

Epoch 1/30
357/357 [==============================] - 1s 2ms/step - loss: 72224.5444
Epoch 2/30
357/357 [==============================] - 0s 218us/step - loss: 71992.0166
Epoch 3/30
357/357 [==============================] - 0s 221us/step - loss: 70659.0451
Epoch 4/30
357/357 [==============================] - 0s 230us/step - loss: 63925.9313
Epoch 5/30
357/357 [==============================] - 0s 222us/step - loss: 58293.8823
Epoch 6/30
357/357 [==============================] - 0s 244us/step - loss: 55859.2495
Epoch 7/30
357/357 [==============================] - 0s 215us/step - loss: 52453.2293
Epoch 8/30
357/357 [==============================] - 0s 220us/step - loss: 46943.9290
Epoch 9/30
357/357 [==============================] - 0s 241us/step - loss: 39052.7432
Epoch 10/30
357/357 [==============================] - 0s 224us/step - loss: 27625.8793
Epoch 11/30
357/357 [==============================] - 0s 221us/step - loss: 14400.5311
Epoch 12/30
357/357 [======================

In [61]:
model_lstm_dish2 = Sequential()
model_lstm_dish2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_dish2.add(LSTM(50, activation='relu'))
model_lstm_dish2.add(Dense(1))
model_lstm_dish2.compile(optimizer='adam', loss='mse')

model_lstm_dish2.fit(LSTM_X_dish2, LSTM_y_dish2, epochs=30, verbose=1)

Epoch 1/30
317/317 [==============================] - 1s 2ms/step - loss: 1215228.4507
Epoch 2/30
317/317 [==============================] - 0s 225us/step - loss: 1206350.4531
Epoch 3/30
317/317 [==============================] - 0s 224us/step - loss: 1161196.1408
Epoch 4/30
317/317 [==============================] - 0s 239us/step - loss: 996442.9509
Epoch 5/30
317/317 [==============================] - 0s 255us/step - loss: 744306.2946
Epoch 6/30
317/317 [==============================] - 0s 250us/step - loss: 690677.4789
Epoch 7/30
317/317 [==============================] - 0s 230us/step - loss: 649261.3214
Epoch 8/30
317/317 [==============================] - 0s 228us/step - loss: 604774.0743
Epoch 9/30
317/317 [==============================] - 0s 245us/step - loss: 548794.3616
Epoch 10/30
317/317 [==============================] - 0s 227us/step - loss: 477436.6438
Epoch 11/30
317/317 [==============================] - 0s 242us/step - loss: 386654.3752
Epoch 12/30
317/317 [========

In [62]:
model_lstm_micr = Sequential()
model_lstm_micr.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_micr.add(LSTM(50, activation='relu'))
model_lstm_micr.add(Dense(1))
model_lstm_micr.compile(optimizer='adam', loss='mse')

model_lstm_micr.fit(LSTM_X_micr, LSTM_y_micr, epochs=30, verbose=1)

Epoch 1/30
275/275 [==============================] - 1s 2ms/step - loss: 1673885.9345
Epoch 2/30
275/275 [==============================] - 0s 225us/step - loss: 1671558.7100
Epoch 3/30
275/275 [==============================] - 0s 227us/step - loss: 1663022.2364
Epoch 4/30
275/275 [==============================] - 0s 232us/step - loss: 1626515.8068
Epoch 5/30
275/275 [==============================] - 0s 260us/step - loss: 1496828.6950
Epoch 6/30
275/275 [==============================] - 0s 278us/step - loss: 1447157.4364
Epoch 7/30
275/275 [==============================] - 0s 230us/step - loss: 1410394.6150
Epoch 8/30
275/275 [==============================] - 0s 228us/step - loss: 1392839.5041
Epoch 9/30
275/275 [==============================] - 0s 262us/step - loss: 1369172.3968
Epoch 10/30
275/275 [==============================] - 0s 303us/step - loss: 1322857.6523
Epoch 11/30
275/275 [==============================] - 0s 292us/step - loss: 1255722.8064
Epoch 12/30
275/275 [

In [63]:
model_lstm_refg = Sequential()
model_lstm_refg.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_lstm, n_features_lstm)))
model_lstm_refg.add(LSTM(50, activation='relu'))
model_lstm_refg.add(Dense(1))
model_lstm_refg.compile(optimizer='adam', loss='mse')



model_lstm_refg.fit(LSTM_X_refg, LSTM_y_refg, epochs=30, verbose=1)



Epoch 1/30
4485/4485 [==============================] - 2s 350us/step - loss: 18809.0217
Epoch 2/30
4485/4485 [==============================] - 1s 210us/step - loss: 1485.7068
Epoch 3/30
4485/4485 [==============================] - 1s 210us/step - loss: 1462.6039
Epoch 4/30
4485/4485 [==============================] - 1s 226us/step - loss: 1451.3930
Epoch 5/30
4485/4485 [==============================] - 1s 211us/step - loss: 1482.3535
Epoch 6/30
4485/4485 [==============================] - 1s 222us/step - loss: 1457.6663
Epoch 7/30
4485/4485 [==============================] - 1s 212us/step - loss: 1461.1083
Epoch 8/30
4485/4485 [==============================] - 1s 239us/step - loss: 1452.6032
Epoch 9/30
4485/4485 [==============================] - 1s 228us/step - loss: 1471.2911
Epoch 10/30
4485/4485 [==============================] - 1s 229us/step - loss: 1457.6867
Epoch 11/30
4485/4485 [==============================] - 1s 213us/step - loss: 1446.9150
Epoch 12/30
4485/4485 [======

In [0]:
def CNN_pre_processing_test (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  x = dataset.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  dataset1 = pd.DataFrame(x_scaled)


  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset1[0][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)



  test_array = []
  for i in range(0,len(final_array)):
      temp = []
      for j in range(0,10):
          temp.append([0,0])
      for j in range(0,len(final_array[i])):
          temp.append(final_array[i][j])
      for j in range(0,10):
          temp.append([0,0])
      test_array.append(temp)



  training_set = []
  for i in range(0,len(test_array)):
      for j in range(0,len(test_array[i])-20):
          x_y = []
          temp = []
          for k in range(j,j+20):
              temp.append(test_array[i][k][0])
          x_y.append(temp)
          x_y.append(test_array[i][j+10][1])
          training_set.append(x_y)


  X = []
  y = []
  for i in range(0,len(training_set)):
    lets = training_set[i][0]
    lets = np.asarray(lets)
    X.append(lets)
    y.append(training_set[i][1])


  X = np.asarray(X)
  y = np.asarray(y)
  from keras.utils import to_categorical
  y = to_categorical(y)



  return X,y;

In [0]:
def CNN_pre_processing_finaly (dataset, appliance, starting, ending, value1, value2):
  for i in range(starting, ending):
    if value1 == 0:
      if dataset[appliance][i] > value2:
        dataset["Lite"][i] = 1
      else:
        dataset["Lite"][i] = 0

    else:
      if dataset[appliance][i] > value1:
        if dataset[appliance][i] < value2:
          dataset["Lite"][i] = 1
        else:
          dataset["Lite"][i] = 0
      else:
        dataset["Lite"][i] = 0

  lite = (dataset.index[starting+1] - dataset.index[starting])
  lite = str(lite)
  main_array = []
  check = lite

  temp = []
  for i in range(starting+1,ending):
      difference = dataset.index[i] - dataset.index[i-1]
      string = str(difference)
      if string == check:
          temp1 = []
          temp2 = []
          temp1.append(dataset[appliance][i])
          temp1.append(dataset["Lite"][i])
          temp.append(temp1)
      else:
          main_array.append(temp)
          temp = []



  final_array = []
  for i in range(0,len(main_array)):
      length = len(main_array[i])
      modu = length%20
      if modu < 10:
          temp = []
          for j in range(0,len(main_array[i])-modu):
              temp.append(main_array[i][j])
          final_array.append(temp)
      else:
          temp = main_array[i]
          for j in range(0,20 - modu):
              array = [0,0]
              temp.append(array)
          final_array.append(temp)


  final_y = []
  for i in range(0,len(final_array)):
    for j in range(0,len(final_array[i])):
      final_y.append(final_array[i][j][0])
  len(final_y)

  return final_y;

In [0]:
X_test_refg, y_test_refg = CNN_pre_processing_test(dataset, "Refg", (len(dataset.index)-8000), len(dataset.index), 0, 100)

X_test_micr, y_test_micr = CNN_pre_processing_test(dataset, "Micr", (len(dataset.index)-8000), len(dataset.index), 0, 600)

X_test_dish1, y_test_dish1 = CNN_pre_processing_test(dataset, "Dish", (len(dataset.index)-8000), len(dataset.index), 100, 500)

X_test_dish2, y_test_dish2 = CNN_pre_processing_test(dataset, "Dish", (len(dataset.index)-8000), len(dataset.index), 0, 850)


In [0]:
final_y = CNN_pre_processing_finaly(dataset, "Refg",  (len(dataset.index)-8000), len(dataset.index), 0, 100)

final_y_micr = CNN_pre_processing_finaly(dataset, "Micr", (len(dataset.index)-8000), len(dataset.index), 0, 600)

final_y_dish1 = CNN_pre_processing_finaly(dataset, "Dish", (len(dataset.index)-8000), len(dataset.index), 100, 500)

final_y_dish2 = CNN_pre_processing_finaly(dataset, "Dish", (len(dataset.index)-8000), len(dataset.index), 0, 850)

In [68]:
print(len(X_test_refg), len(y_test_refg), len(final_y))
print(len(X_test_micr), len(y_test_micr), len(final_y_micr))
print(len(X_test_dish1), len(y_test_dish1), len(final_y_dish1))
print(len(X_test_dish2), len(y_test_dish2), len(final_y_dish2))

7980 7980 7980
7980 7980 7980
7980 7980 7980
7980 7980 7980


In [0]:
def testset_lstm (lite, deadload):
  qwerty = []
  flag = 0
  for i in range(0,len(lite)):
    if lite[i] == 0:
      qwerty.append([deadload])
      flag = 0
      temp = [0,0,0,0,0]
    else:
      flag+=1
      temp2 = []
      qwerty.append(temp)
      temp.append(flag)
      for j in range(len(temp)-5,len(temp)):
        temp2.append(temp[j])
      temp = temp2
  
  for i in range(0,len(qwerty)):
    if len(qwerty) > 2:
      good = []
      for j in range(0,len(qwerty[i])-1):
        good.append(qwerty[i][j])
      qwerty[i] = good
    else:
      qwerty[i] = qwerty[i].append(deadload)


  final = []
  for i in range(0,len(qwerty)):
    if len(qwerty[i]) > 2:
      good = []
      for j in range(0,len(qwerty[i])):
        good.append(qwerty[i][j])
      final.append(good)
    else:
      temp = []
      temp.append(deadload)
      final.append(temp)

  return final;


In [0]:
lite_refg = []
for i in range(0,len(X_test_refg)):
  x_input = X_test_refg[i].reshape(1,20,1)
  yhat = model_refg.predict_classes(x_input,verbose=0)
  lite_refg.append(yhat[0])

In [0]:
lite_micr = []
for i in range(0,len(X_test_micr)):
  x_input = X_test_micr[i].reshape(1,20,1)
  yhat = model_micr.predict_classes(x_input,verbose=0)
  lite_micr.append(yhat[0])

In [0]:
lite_dish1 = []
for i in range(0,len(X_test_dish1)):
  x_input = X_test_dish1[i].reshape(1,20,1)
  yhat = model_dish1.predict_classes(x_input,verbose=0)
  lite_dish1.append(yhat[0])

In [0]:
lite_dish2 = []
for i in range(0,len(X_test_dish2)):
  x_input = X_test_dish2[i].reshape(1,20,1)
  yhat = model_dish2.predict_classes(x_input,verbose=0)
  lite_dish2.append(yhat[0])

In [0]:
Refg = testset_lstm(lite_refg, 7.5)
Micr = testset_lstm(lite_micr, 4)
Dish1 = testset_lstm(lite_dish1, 0)
Dish2 = testset_lstm(lite_dish2, 0)

In [0]:
#

In [0]:
lol = 0
error = 0
total = 0
tot = 0
tot_m = 0
total_m = 0
tot_d = 0
total_d = 0

for i in range(0,len(Refg)):
  if len(Refg[i]) < 2:
    predic = Refg[i][0]
  else:
    x_input = np.asarray(Refg[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_refg.predict(x_input, verbose=0)
    predic = yhat[0][0]
  #print(predic, final_y[i])
  tot += predic
  total += final_y[i]
  #lol = (abs(predic+0.1 - final_y[i]+0.1)/ (final_y[i]+0.1))/7980




for i in range(0,len(Micr)):
  if len(Micr[i]) < 2:
    predic = Micr[i][0]
  else:
    x_input = np.asarray(Micr[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_micr.predict(x_input, verbose=0)
    predic = yhat[0][0]
  #print(predic, final_y_micr[i])
  tot_m += predic
  total_m += final_y_micr[i]
  #lol = (abs(predic+0.1 - final_y[i]+0.1)/ (final_y[i]+0.1))/7980




for i in range(0,len(Dish2)):

  if len(Dish2[i]) < 2:
    if len(Dish1[i]) < 2:
      predic = Dish2[i][0]
    else:
      x_input = np.asarray(Dish1[i])
      x_input = x_input.reshape((1,5,1))
      yhat = model_lstm_dish1.predict(x_input, verbose=0)
      predic = yhat[0][0]
  else:
    x_input = np.asarray(Dish2[i])
    x_input = x_input.reshape((1,5,1))
    yhat = model_lstm_dish2.predict(x_input, verbose=0)
    predic = yhat[0][0]

  tot_d += predic
  total_d += final_y_dish2[i]

  



In [77]:
print(tot, total)
print(tot_m, total_m)
print(tot_d, total_d)


446497.73806762695 462011.2909343933
139598.81091308594 129298.89045545923
178623.67350769043 205209.59031777052


In [0]:
erro = abs((total + total_m + total_d) - (tot + tot_m + tot_d))/(total + total_m + total_d)

In [79]:
((1 - (abs(tot_d - total_d)/ total_d)) + (1 - (abs(tot_m - total_m)/ total_m)) + (1 - (abs(tot - total)/ total)))/3

0.919068994733307

In [80]:
(1-erro)*100

96.00768865397453